<a href="https://colab.research.google.com/github/JAYESH2505/Brain_Tumor_CNN/blob/main/brain_tumor_resnet_app_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install streamlit -q

In [28]:
!wget -qO - https://ipv4.icanhazip.com

34.125.236.210


In [33]:
%%writefile app.py
import streamlit as st
from PIL import Image
import torch
from torchvision import models, transforms

# Load the fine-tuned ResNet model
@st.cache_resource
def load_model():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # Check if CUDA is available
    model = models.resnet18(pretrained=False)  # Initialize ResNet18
    model.fc = torch.nn.Linear(model.fc.in_features, 4)  # Adjust for 4 classes
    model.load_state_dict(torch.load("brain_tumor_model.pth", map_location=device))  # Map the model to the appropriate device
    model.to(device)  # Send the model to the correct device
    model.eval()  # Set the model to evaluation mode
    return model

# Initialize the model
model = load_model()

# Define image transformations (same as used in training)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])



# Define class labels
class_names = ['glioma', 'healthy', 'meningioma', 'pituitary']

# Streamlit interface
st.title("Brain Tumor Classification with ResNet")
st.write("Upload an MRI image to classify the type of brain tumor.")


# File uploader
uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "png", "jpeg"])

if uploaded_file is not None:
    # Display the uploaded image
    image = Image.open(uploaded_file).convert("RGB")
    st.image(image, caption="Uploaded Image", use_column_width=True)

    # Transform the image
    image_tensor = transform(image).unsqueeze(0)  # Add batch dimension

    # Make a prediction
    with torch.no_grad():
        output = model(image_tensor)
        _, predicted_class = torch.max(output, 1)

    # Display the prediction
    st.write(f"Prediction: **{class_names[predicted_class.item()]}**")

Overwriting app.py


In [35]:
%%writefile app.py
import streamlit as st
from PIL import Image
import torch
from torchvision import models, transforms

# Ensure page config is set as the first Streamlit command
st.set_page_config(page_title="Brain Tumor Classifier", layout="wide")

# 1. Load the fine-tuned ResNet model
@st.cache_resource
def load_model():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # Check if CUDA is available
    model = models.resnet18(pretrained=False)  # Initialize ResNet18
    model.fc = torch.nn.Linear(model.fc.in_features, 4)  # Adjust for 4 classes
    model.load_state_dict(torch.load("brain_tumor_model.pth", map_location=device))  # Map the model to the appropriate device
    model.to(device)  # Send the model to the correct device
    model.eval()  # Set the model to evaluation mode
    return model

# Initialize the model
model = load_model()

# 2. Define image transformations (same as used in training)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# 3. Define class labels
class_names = ['glioma', 'healthy', 'meningioma', 'pituitary']

# 4. Streamlit interface
st.sidebar.title("Upload Section")
st.sidebar.write("Upload an MRI image or video for classification:")

# Sidebar for file upload
uploaded_file = st.sidebar.file_uploader("Choose an image or video...", type=["jpg", "png", "jpeg", "mp4"])

# Main layout
st.title("🧠 Brain Tumor Classification")
st.write("**Upload an MRI image in the sidebar, and the app will classify the tumor type.**")

# If an image file is uploaded
if uploaded_file is not None and uploaded_file.type.startswith("image"):
    # Display uploaded image
    image = Image.open(uploaded_file).convert("RGB")
    st.image(image, caption="Uploaded Image", use_column_width=True, clamp=True)

    # Transform the image and make predictions
    image_tensor = transform(image).unsqueeze(0)  # Add batch dimension
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    image_tensor = image_tensor.to(device)

    with torch.no_grad():
        output = model(image_tensor)
        _, predicted_class = torch.max(output, 1)

    # Display the prediction
    st.markdown(
        f"<h3 style='text-align: center; color: green;'>Prediction: {class_names[predicted_class.item()]}</h3>",
        unsafe_allow_html=True,
    )

# If a video file is uploaded (currently not processed)
elif uploaded_file is not None and uploaded_file.type.startswith("video"):
    st.video(uploaded_file)
    st.write("**Video upload is currently for display only. Image classification is available.**")

else:
    st.info("Upload an MRI image in the sidebar to start the classification.")


Overwriting app.py


In [36]:
!streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.236.210:8501

your url is: https://wild-cycles-act.loca.lt
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/content/app.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (

In [19]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 97.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 12.6 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [20]:
import gradio as gr
from PIL import Image
import torch
from torchvision import models, transforms

# 1. Load the fine-tuned ResNet model
def load_model():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # Check for GPU
    model = models.resnet18(pretrained=False)  # Initialize ResNet18
    model.fc = torch.nn.Linear(model.fc.in_features, 4)  # Modify for 4 classes
    model.load_state_dict(torch.load("brain_tumor_model.pth", map_location=device))  # Load weights
    model.to(device)  # Send model to device
    model.eval()  # Evaluation mode
    return model

# Load the model
model = load_model()

# 2. Define image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize image
    transforms.ToTensor(),          # Convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])

# 3. Define class labels
class_names = ['glioma', 'healthy', 'meningioma', 'pituitary']

# 4. Define the prediction function
def predict_image(image):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Transform the image
    image = transform(image).unsqueeze(0)  # Add batch dimension
    image = image.to(device)  # Send to device

    with torch.no_grad():  # Disable gradients
        output = model(image)  # Predict
        _, predicted_class = torch.max(output, 1)  # Get class index

    return class_names[predicted_class.item()]  # Return class name

# 5. Set up the Gradio interface
interface = gr.Interface(
    fn=predict_image,  # Prediction function
    inputs=gr.Image(type="pil"),  # Input type is an image
    outputs="text",  # Output is text
    title="Brain Tumor Classification",  # Title for the app
    description="Upload an MRI image to classify the type of brain tumor into one of the following categories: glioma, healthy, meningioma, or pituitary."
)

# 6. Launch the Gradio interface
interface.launch(share=True)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
<ipython-input-20-685d9375dd60>:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the function

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4eadee4fd295355dbc.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
